# LoC image analysis

This notebook is designed to take the raw output of a LoC data set, monolayer or bilayer, and:
 (trying to check if dask multichan works)

5. Localise and measure properties
6. Unite the localisations over the z-range
7. Save out the Z-tracks
8. Extract the maximum intensity from each cell

In [1]:
import os
import glob
from octopusheavy import DaskOctopusHeavyLoader
import napari
from skimage.io import imshow,  imsave, imread
import napari
import btrack
from tqdm.auto import tqdm
import dask.array as da

# Loading images

Define root path and individual experiment IDs

In [2]:
root_path = '/run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/shared/Lung on chip/Light microscopy'

In [3]:
expt_IDs = ['co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-SPC-PDPN-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60_/images/',
            'co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-VWF-iCAM1-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_70_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-AQP5-proSPC-ZO1/Day7_static/_20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_150_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-AQP5-proSPC-ZO1/Day14_static/_20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_262_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-CAV1-proSPC-ZO1/Day7_static/_20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_130_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-CAV1-proSPC-ZO1/Day14_static/_20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_253_/images/'
           ]

## Check to see what channels each expt has

In [4]:
for i, expt in enumerate(expt_IDs):
    images = DaskOctopusHeavyLoader(os.path.join(root_path, expt), remove_background=False)
    if 'MASK1' in [channel.name for channel in images.channels]:
        images = DaskOctopusHeavyLoader(os.path.join(root_path, expt), remove_background=False)
        print(i, images.channels)

0 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
1 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
2 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
3 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
4 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
5 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]


In [5]:
images['CH1']

dask.array<stack, shape=(81, 2304, 2304), dtype=uint16, chunksize=(1, 2304, 2304), chunktype=numpy.ndarray>

### Defining properties to measure similarities of z-slices

In [6]:
props = ('axis_major_length', 
         'axis_minor_length', 
         'eccentricity', 
         'area', 
         #'intensity_image', 
         'orientation',
         'mean_intensity')

# Retracking hte first coculture

In [7]:
for i, expt in tqdm(enumerate(expt_IDs), total = len(expt_IDs)):
    props = ('axis_major_length', 
         'axis_minor_length', 
         'eccentricity', 
         'area', 
         #'intensity_image', 
         'orientation',
         'mean_intensity')
    images = DaskOctopusHeavyLoader(os.path.join(root_path, expt), remove_background=False)
    multichannel_stack = da.stack([images['CH1'], images['CH2'], images['CH3'], images['CH4']], axis = -1)
    if 'MASK1' in [channel.name for channel in images.channels]: 
        ### create filename for tracks and objects 
        dirname = os.path.dirname(images.files('CH1')[0]).replace('images','')
        objects_fn = os.path.join(dirname, 'objects.hdf5')
        tracks_fn = os.path.join(dirname, 'tracks.hdf5')
        ### try saving locally instead of online
        tracks_fn = tracks_fn.split('Light microscopy/')[-1].replace('/', '_')
        objects = btrack.utils.segmentation_to_objects(
            images['MASK1'], 
            multichannel_stack,
            properties = props,#('area', 'mean_intensity', 'intensity_image'), 
            use_weighted_centroid = False
        )
        ### prune objects 
        objects = [o for o in objects if o.properties['area']>50]
#         ### save out objects
#         with btrack.dataio.HDF5FileHandler(
#              objects_fn, 'w', obj_type='obj_type_1',
#         ) as hdf:
#             hdf.write_segmentation(images['MASK1'])
#             hdf.write_objects(objects)
        ### redefine properties as multichannel image was measured
        props = ('axis_major_length', 
             'axis_minor_length', 
             'eccentricity', 
             'area', 
             #'intensity_image', 
             'orientation',
             'mean_intensity-0',
             'mean_intensity-1',
             'mean_intensity-2',
             'mean_intensity-3',)
        
        # initialise a tracker session using a context manager
        with btrack.BayesianTracker() as tracker:
            # configure the tracker using a config file
            tracker.configure('/home/dayn/analysis/BayesianTracker/models/particle_config.json')
            tracker.verbose = True
            ### set max search radius
            tracker.max_search_radius = 100
            # use visual features to track
            tracker.features = props
            # append the objects to be tracked
            tracker.append(objects)
            # set the volume (Z axis volume limits default to [-1e5, 1e5] for 2D data)
            tracker.volume=((0, 2304), (0, 2304), (-1e5, 1e5))
            # track them (in interactive mode)
#             tracker.track_interactive(step_size=100)
            tracker.track(tracking_updates =['visual', 'motion'], step_size=10)
            # generate hypotheses and run the global optimizer
            tracker.optimize()
            # get the tracks as a python list
            tracks = tracker.tracks
#             # filter tracks
#             tracks = [track for track in tracks if len(track) >= 3]
            # optional: get the data in a format for napari
            #   data, properties, graph = tracker.to_napari()
            tracker.export('test_cocult.hdf5', obj_type = 'obj_type_1')
    break

  0%|          | 0/6 [00:00<?, ?it/s]

[INFO][2022/12/02 11:29:08 AM] Localizing objects from segmentation...
[INFO][2022/12/02 11:29:08 AM] Found intensity_image data
[INFO][2022/12/02 11:31:23 AM] Objects are of type: <class 'dict'>
[INFO][2022/12/02 11:31:24 AM] ...Found 70496 objects in 81 frames.
[INFO][2022/12/02 11:31:24 AM] Loaded btrack: /home/dayn/analysis/BayesianTracker/btrack/libs/libtracker.so
[INFO][2022/12/02 11:31:24 AM] btrack (v0.5.0) library imported
[INFO][2022/12/02 11:31:24 AM] Starting BayesianTracker session
[INFO][2022/12/02 11:31:24 AM] Loading configuration file: /home/dayn/analysis/BayesianTracker/models/particle_config.json
[INFO][2022/12/02 11:31:24 AM] Setting max_search_radius -> 100
[INFO][2022/12/02 11:31:24 AM] Setting features -> ('axis_major_length', 'axis_minor_length', 'eccentricity', 'area', 'orientation', 'mean_intensity-0', 'mean_intensity-1', 'mean_intensity-2', 'mean_intensity-3')
[INFO][2022/12/02 11:31:24 AM] Objects are of type: <class 'list'>
[INFO][2022/12/02 11:31:26 AM] Se

GLPK Integer Optimizer 5.0
46352 rows, 41159 columns, 59142 non-zeros
41159 integer variables, all of which are binary
Preprocessing...
23176 rows, 41159 columns, 59142 non-zeros
41159 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 23176
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
23176 rows, 41159 columns, 59142 non-zeros
*     0: obj =   1.470728152e+05 inf =   0.000e+00 (12076)
Perturbing LP to avoid stalling [1021]...
*  5623: obj =   1.049431084e+05 inf =   0.000e+00 (4372)
* 10489: obj =   9.041514166e+04 inf =   0.000e+00 (209) 7
Removing LP perturbation [10714]...
* 10714: obj =   9.037771352e+04 inf =   0.000e+00 (0) 1
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+ 10714: mip =     not found yet >=              -inf        (1; 0)
+ 10714: >>>>>   9.0377713

[WARNING][2022/12/02 11:41:33 AM] Changing HDF filename to co-culture_iVECs+iAT2AT1_Folder_20220808_A2-A5_analysis_20221125_DAPI-SPC-PDPN-ZO1__20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60__tracks.h5
[INFO][2022/12/02 11:41:33 AM] Opening HDF file: co-culture_iVECs+iAT2AT1_Folder_20220808_A2-A5_analysis_20221125_DAPI-SPC-PDPN-ZO1__20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60__tracks.h5...
[INFO][2022/12/02 11:41:34 AM] Writing tracks/obj_type_1
[WARNING][2022/12/02 11:41:34 AM] Removing tracks/obj_type_1.
[INFO][2022/12/02 11:41:34 AM] Writing dummies/obj_type_1
[INFO][2022/12/02 11:41:34 AM] Writing LBEP/obj_type_1
[INFO][2022/12/02 11:41:34 AM] Writing fates/obj_type_1
[INFO][2022/12/02 11:41:34 AM] Closing HDF file: co-culture_iVECs+iAT2AT1_Folder_20220808_A2-A5_analysis_20221125_DAPI-SPC-PDPN-ZO1__20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60__tracks.h5
[INFO][2022/12/02 11:41:34 AM] Ending BayesianTracker 

In [9]:
tracks_fn

'co-culture_iVECs+iAT2AT1_Folder_20220808_A2-A5_analysis_20221125_DAPI-SPC-PDPN-ZO1__20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60__tracks.hdf5'

In [22]:
tracks[0]

,ID,t,x,y,z,parent,root,state,generation,dummy,mean_intensity-3,axis_minor_length,orientation,mean_intensity-1,mean_intensity-0,mean_intensity-2,eccentricity,axis_major_length,area
0,93,0,1128.883333,3.766667,0.0,93,93,5,0,False,"(1,) array","(1,) array","(1,) array","(1,) array","(1,) array","(1,) array","(1,) array","(1,) array","(1,) array"


# It won't let me save tracks to the server? But will let me save locally

In [18]:
tracks_fn = tracks_fn.replace('h5', 'hdf5')

In [16]:
tracks[0]

,ID,t,x,y,z,parent,root,state,generation,dummy,mean_intensity-3,axis_minor_length,orientation,mean_intensity-1,mean_intensity-0,mean_intensity-2,eccentricity,axis_major_length,area
0,93,0,1128.883333,3.766667,0.0,93,93,5,0,False,"(1,) array","(1,) array","(1,) array","(1,) array","(1,) array","(1,) array","(1,) array","(1,) array","(1,) array"


In [25]:
with btrack.dataio.HDF5FileHandler('test.h5', "w", obj_type="obj_type_1") as hdf:
    hdf.write_tracks(tracks)

[INFO][2022/12/02 11:53:24 AM] Opening HDF file: test.h5...
[INFO][2022/12/02 11:53:25 AM] Writing objects/obj_type_1
[INFO][2022/12/02 11:53:25 AM] Writing labels/obj_type_1
[INFO][2022/12/02 11:53:25 AM] Loading objects/obj_type_1 (59338, 5) (59338 filtered: None)
[INFO][2022/12/02 11:53:26 AM] Writing properties/obj_type_1/axis_major_length (59338,)
[INFO][2022/12/02 11:53:26 AM] Writing properties/obj_type_1/axis_minor_length (59338,)
[INFO][2022/12/02 11:53:26 AM] Writing properties/obj_type_1/eccentricity (59338,)
[INFO][2022/12/02 11:53:26 AM] Writing properties/obj_type_1/area (59338,)
[INFO][2022/12/02 11:53:26 AM] Writing properties/obj_type_1/orientation (59338,)
[INFO][2022/12/02 11:53:26 AM] Writing properties/obj_type_1/mean_intensity-0 (59338,)
[INFO][2022/12/02 11:53:26 AM] Writing properties/obj_type_1/mean_intensity-1 (59338,)
[INFO][2022/12/02 11:53:26 AM] Writing properties/obj_type_1/mean_intensity-2 (59338,)
[INFO][2022/12/02 11:53:26 AM] Writing properties/obj_ty

In [20]:
with btrack.dataio.HDF5FileHandler(tracks_fn, "r", obj_type="obj_type_1") as hdf:
    ids = hdf.tracks

[INFO][2022/12/02 11:49:24 AM] Opening HDF file: co-culture_iVECs+iAT2AT1_Folder_20220808_A2-A5_analysis_20221125_DAPI-SPC-PDPN-ZO1__20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60__tracks.hdf5...
[INFO][2022/12/02 11:49:24 AM] Loading tracks/obj_type_1
[INFO][2022/12/02 11:49:24 AM] Loading LBEP/obj_type_1
[INFO][2022/12/02 11:49:24 AM] Loading objects/obj_type_1 (59338, 5) (59338 filtered: None)
[INFO][2022/12/02 11:49:25 AM] Closing HDF file: co-culture_iVECs+iAT2AT1_Folder_20220808_A2-A5_analysis_20221125_DAPI-SPC-PDPN-ZO1__20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60__tracks.hdf5


NameError: free variable 'dummy_obj' referenced before assignment in enclosing scope

#### testing unique fn 

In [49]:
tracks_fn.split('Light microscopy/')[-1].replace('/', '_')

'co-culture_iVECs+iAT2AT1_Folder_20220808_A2-A5_analysis_20221125_DAPI-SPC-PDPN-ZO1__20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60__tracks.hdf5'

In [37]:
tracks_fn

'/run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/shared/Lung on chip/Light microscopy/co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-SPC-PDPN-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60_/tracks.hdf5'

In [43]:
test_tracks_fn = '/run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/users/dayn/test_tracks_output/tracks.hdf5'

In [44]:
with btrack.dataio.HDF5FileHandler(test_tracks_fn, "w", obj_type="obj_type_1") as hdf:
    hdf.write_tracks(tracks)

[INFO][2022/12/02 09:25:39 AM] Opening HDF file: /run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/users/dayn/test_tracks_output/tracks.hdf5...
[INFO][2022/12/02 09:25:39 AM] Writing objects/obj_type_1
[INFO][2022/12/02 09:25:39 AM] Writing labels/obj_type_1
[INFO][2022/12/02 09:25:39 AM] Loading objects/obj_type_1 (59306, 5) (59306 filtered: None)
[INFO][2022/12/02 09:25:41 AM] Writing properties/obj_type_1/axis_major_length (59306,)
[INFO][2022/12/02 09:25:41 AM] Closing HDF file: /run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/users/dayn/test_tracks_output/tracks.hdf5


OSError: [Errno 95] Unable to extend file properly, errno = 95, error message = 'operation not supported' (file write failed: time = Fri Dec  2 09:25:41 2022
, filename = '/run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/users/dayn/test_tracks_output/tracks.hdf5', file descriptor = 66, errno = 95, error message = 'Operation not supported', buf = 0x5620987ad0c8, total write size = 3368, bytes this sub-write = 3368, bytes actually written = 18446744073709551615, offset = 0)

## Checking tracks prior to plotting single cell info

In [ ]:
import napari

In [ ]:
viewer = napari.Viewer()

viewer.add_image(ch0, colormap='blue', blending = 'additive')
viewer.add_image(ch3, colormap= 'gray', blending = 'additive')
viewer.add_image(ch1, colormap='red', blending = 'additive')
viewer.add_image(ch2, colormap= 'green', blending = 'additive')

viewer.add_labels(mask_stack)

viewer.add_tracks(ch1_data)
viewer.add_tracks(ch2_data)
